In [ ]:
#used to import data
import pandas as pd
#Evaluates the model
from sklearn.metrics import log_loss
import time #just used to see how long the model takes to run
from pytorch_tabnet.tab_model import TabNetRegressor

In [ ]:
df_11 = pd.read_csv('data/pbp/ready_to_model/10_11_pbp_cleaned.csv')
df_12 = pd.read_csv('data/pbp/ready_to_model/11_12_pbp_cleaned.csv')
df_13 = pd.read_csv('data/pbp/ready_to_model/12_13_pbp_cleaned.csv')
df_14 = pd.read_csv('data/pbp/ready_to_model/13_14_pbp_cleaned.csv')
df_15 = pd.read_csv('data/pbp/ready_to_model/14_15_pbp_cleaned.csv')
df_16 = pd.read_csv('data/pbp/ready_to_model/15_16_pbp_cleaned.csv')
df_17 = pd.read_csv('data/pbp/ready_to_model/16_17_pbp_cleaned.csv')
df_18 = pd.read_csv('data/pbp/ready_to_model/17_18_pbp_cleaned.csv')
df_19 = pd.read_csv('data/pbp/ready_to_model/18_19_pbp_cleaned.csv')
df_20 = pd.read_csv('data/pbp/ready_to_model/19_20_pbp_cleaned.csv')
df_21 = pd.read_csv('data/pbp/ready_to_model/20_21_pbp_cleaned.csv')
df = pd.concat([df_11,df_12,df_13,df_14,df_15,df_16,df_17,df_18,df_19,df_20,df_21])

In [ ]:
def get_X_y(df):
    ''' This function takes in a dataframe and splits it into the X and y variables
    '''
    X = df.drop(['is_goal'], axis=1).to_numpy()
    y = df.is_goal.to_numpy()
    
    return X,y

In [ ]:
X,y = get_X_y(df)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=101)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.50, random_state=101)

In [ ]:
#X_train.values
y_train = y_train.reshape(-1,1)
y_val = y_val.reshape(-1,1)
y_test = y_test.reshape(-1,1)

In [ ]:
reg = TabNetRegressor()
model = reg.fit(X_train, y_train, 
                eval_set = [(X_train, y_train), (X_val, y_val)],
                eval_name=['train','val'],
                eval_metric = ['logloss'],                
                max_epochs = 20)

In [ ]:
preds = reg.predict(X_test)
print(log_loss(y_test, preds))

In [ ]:
#after 10 epochs logloss = .197
#after 10 epochs logloss = .197
#Added a validation set, 10 epochs logloss = .198
#With the validation set and 10 epochs, logloss = .1965 <--- slight improvement